## Initial data solver using relaxation

We try to solve the time symmetric initial data when given a $ρ$ field on a cube. 
We put Robin boudary conditions.

Thus we shall be solving:

$$
ψ_{tt} = \Delta ψ + ψ^5 ρ \;\;\;\; \vec{x}\cdot \nabla ψ + (1-ψ) = 0
$$

Using second order finite differences inside the domain and first order at the boudaries we get for the boundary values (which are not evolved):

For boundary conditions we shall put an outgoing wave, but taking into account that we want a radial wave and that something which at first order should be a monopolar field.

$$
ψ_t = (x^i\partial_i ψ - (ψ - ψ_0))/r
$$ 
Same for the time derivative, but that part should be unnessary becuse this should decay to a static solution.

The solution we are seeking is of the form of an outgoing wave plus a static solution, the one we are after. It well behave as a monopole at large distance, plus a constant.

$$
\psi(t,x) =\frac{f(t-r) + M}{r} + \psi_0
$$

Thus we have, 
$$
\partial_t \psi = \frac{f'(t-r)}{r}, \;\;\;\;\; \frac{\vec{x}\cdot \nabla \psi}{r} = -f'/r - f/r^2 - M/r^2 
$$

therefore asymtotically we should have, 
$$ 
\partial_t \psi = - \frac{\vec{x}\cdot \nabla \psi}{r} - (f+M)/r^2 = - \frac{\vec{x}\cdot ∇\psi}{r} - (\psi - \psi_0)/r 
$$

In [1]:
using SummationByPartsOperators
using Plots
using Revise
using Distributions
using FFTW
using LinearAlgebra
using Base.Threads
using HDF5
 
includet("../PIC/PIC-1D/aux_functions/aux_functions.jl")
includet("local_aux_functions.jl")



First run inidat.ipynb and get the data


In [3]:
data = h5open("data_3d_h_step_cut.h5", "r")

🗂️ HDF5.File: (read-only) data_3d_h_step_cut.h5
├─ 🔢 coord0
├─ 🔢 coord1
├─ 🔢 coord2
├─ 🔢 nvars
├─ 🔢 var0
├─ 🔢 var1
├─ 🔢 var2
├─ 🔢 var3
├─ 🔢 var4
├─ 🔢 var5
└─ 🔢 var6

In [2]:
J_in = size(data["var6"])
Box_x_in = [read(data["coord0"])[1], read(data["coord0"])[end],read(data["coord1"])[1], read(data["coord1"])[end],read(data["coord2"])[1], read(data["coord2"])[end]];
J = (3).*J_in
Box_x = 3*Box_x_in;
dx = differentials(Box_x,J)
V = volume(Box_x)
m = 1
m2 = m^2


Dx = derivative_operator(MattssonNordström2004(), derivative_order = 1, accuracy_order = 2, xmin = Box_x[1], xmax = Box_x[2], N=J[1])
Dy = derivative_operator(MattssonNordström2004(), derivative_order = 1, accuracy_order = 2, xmin = Box_x[3], xmax = Box_x[4], N=J[2])
Dz = derivative_operator(MattssonNordström2004(), derivative_order = 1, accuracy_order = 2, xmin = Box_x[5], xmax = Box_x[6], N=J[3])
D2x = derivative_operator(MattssonNordström2004(), derivative_order = 2, accuracy_order = 2, xmin = Box_x[1], xmax = Box_x[2], N=J[1])
D2y = derivative_operator(MattssonNordström2004(), derivative_order = 2, accuracy_order = 2, xmin = Box_x[3], xmax = Box_x[4], N=J[2])
D2z = derivative_operator(MattssonNordström2004(), derivative_order = 2, accuracy_order = 2, xmin = Box_x[5], xmax = Box_x[6], N=J[3])
x = collect(SummationByPartsOperators.grid(Dx))
y = collect(SummationByPartsOperators.grid(Dy))
z = collect(SummationByPartsOperators.grid(Dz));

In [5]:
π2_in = read(data["var4"])
#heatmap(x,y,π2[:,:,J[3]÷2], aspectratio=1)
∇2_in = read(data["var5"])
#heatmap(x,y,∇2[:,:,J[3]÷2], aspectratio=1)
V_in = read(data["var6"])
#heatmap(x,y,V[:,:,J[3]÷2], aspectratio=1)

π2 = zeros(J...); ∇2 = copy(π2); V = copy(π2);
π2[J_in[1]+1:2J_in[1],J_in[2]+1:2J_in[2],J_in[3]+1,2J_in[3]] = π2_in
∇2[J_in[1]+1:2J_in[1],J_in[2]+1:2J_in[2],J_in[3]+1,2J_in[3]] = ∇2_in
V[J_in[1]+1:2J_in[1],J_in[2]+1:2J_in[2],J_in[3]+1,2J_in[3]] = V_in

"""
Completes with zeros a centered integration region
"""
function enlarge_box(f,Box_x_in, Box_x, J_in, J)
    out = zeros(J...)
    for i in 1:J_in[1]
        for j in 1:J_in[2]
            for k in 1:J_in[3]
                out[J_in[1]+i,J_in[2]+j,J_in[3]+k] = f[i,j,k]
            end
        end
    end
    return out[:,:,:]
end

#heatmap(π2[:,:,(J[3]+1)÷2])

: 

In [5]:
n_fields = 4 # the field and its derivative 
u = zeros(n_fields,J...)
du = similar(u)
d2 = zeros(2,J...);
#dxu = similar(u);
#dyu = similar(u);
#dzu = similar(u);

In [7]:
T = 1000.0
M = 2001
@show dt = T/(M-1)
t = 0.0

#v = zeros(n_fields,M+1,J...)
τ = 1.0 #damping must be positive here
par = (1.0, 1.0, τ) # (a, b, τ)
#par_F = (x,y,z,dxu,dyu,dzu,Dx,Dy,Dz,D2x,D2y,D2z,d2,dx,ρ,J, par)
dxu_x = zeros(2,n_fields,J[2],J[3]); dyu_x = copy(dxu_x); dzu_x = copy(dxu_x)
dxu_y = zeros(2,n_fields,J[1],J[3]); dyu_y = copy(dxu_y); dzu_y = copy(dxu_y)
dxu_z = zeros(2,n_fields,J[1],J[2]); dyu_z = copy(dxu_z); dzu_z = copy(dxu_z)
coors = (x,y,z)
boundary_derivs = (dxu_x,dyu_x,dzu_x,dxu_y,dyu_y,dzu_y,dxu_z,dyu_z,dzu_z)
derivs = (Dx,Dy,Dz,D2x,D2y,D2z)
ρ = (π2,∇2,V)
par_FCR = (coors,boundary_derivs,derivs,d2,dx,ρ,J,n_fields,par)
norms = zeros(n_fields,M)

norms[:,1] = [norm(u[1,:,:,:]),norm(u[2,:,:,:]),norm(u[3,:,:,:]),norm(u[4,:,:,:])]

dt = T / (M - 1) = 0.5


4-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0

In [10]:
for m in 2:M
    RK4_Step!(FCR_Full,u,t,dt,par_FCR)
    t = t + dt
    norms[:,m] = [norm(u[1,:,:,:]),norm(u[2,:,:,:]),norm(u[3,:,:,:]),norm(u[4,:,:,:])]
    if mod(m-1,10)==0
        println("t = $t, norm u_1 = $(norms[1,m]) norm u_2 = $(norms[2,m]), norm u_3 = $(norms[3,m]) norm u_4 = $(norms[4,m])")
    end
    #v[:,m,:,:,:] = u[:,:,:,:]
    
end

LoadError: InterruptException: